### Fletcher Project

In [41]:
import json
import numpy as np
import pandas as pd
import re
import string
from spacy import en
from spacy.en import English, STOPWORDS
import re
import pyLDAvis
from sklearn import cluster
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction import text
from nltk.stem import WordNetLemmatizer 
import numpy as np
from textblob import TextBlob
from sklearn import metrics
from sklearn.externals import joblib

In [22]:
# Export the MongoDB collection to json format and transfer from AWS EC2 to localhost
# mongoexport -d fletcher -c mytweets -o mytweets-bk.json
# scp ubuntu_cloud:/home/jaysips/mytweets-bk.json .
# Read the data into an array
tweets_data = []
tweets_file = open('mytweets-bk.json', 'r')
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue

In [23]:
print(len(tweets_data))

166347


In [24]:
# Get the created_at datetime working
tweetsDF = pd.DataFrame()
tweetsDF['user'] = [tweet['user']['screen_name'] for tweet in tweets_data]
tweetsDF['text'] = [tweet['text'] for tweet in tweets_data]
tweetsDF['location'] = [tweet['place']['country'] if tweet['place'] != None else None 
                        for tweet in tweets_data]
tweetsDF['retweet_count'] = [tweet['retweet_count'] for tweet in tweets_data]

In [26]:
tweetsDF

,user,text,location,retweet_count
0,isabelitabeltr5,U.K.'s Cameron Appeals to the Undecided Ahead ...,None,0
1,TheAusup,"@PuppyMoof @YouTube Stay, PLEASE! I can get EU...",None,0
2,DedieWerner,EU referendum: The view from Colchester https:...,None,0
3,DerekSmith76,RT @BritainUnited1: WHY DOES SNP WANT TO BREAK...,None,0
4,WandisCleavland,U.K.'s Cameron Appeals to the Undecided Ahead ...,None,0
5,RFreeUK_Listen,Now Streaming at https://t.co/V0SeDzX1VE\nleav...,None,0
6,MerleMatthus,U.K.'s Cameron Appeals to the Undecided Ahead ...,None,0
7,matthewdrop,eu hzkzuxkxukzux https://t.co/kMCXKvywzU,None,0
8,elizabethregina,RT @NatalieMcGarry: Last thought before bed. T...,None,0
9,kelry2106,Eu inteligente disse good good,None,0


In [69]:
pd.unique(tweetsDF.location.ravel())

array([None, 'New Zealand', 'United Kingdom', 'Australia', 'भारत',
       'Nederland', 'United States', 'Canada', 'België', 'France',
       'Portugal', 'Nigeria', 'Isle of Man', 'South Africa', 'Ireland',
       'Angola', 'Colombia', 'Italia', 'Danmark', 'Norge', 'España',
       'México', 'Deutschland', 'Gibraltar', 'Ελλάς', 'Mongolia',
       'Malaysia', 'République du Rwanda', 'Saint Lucia', '中华人民共和国',
       'سلطنة عمان', 'ישראל', 'Sverige', 'Österreich', 'Cambodia',
       'Hrvatska', 'Suomi', 'Schweiz', 'الامارات العربية المتحدة',
       'ประเทศไทย', 'България', 'Bangladesh', 'Luxembourg', 'Україна',
       'Polska', 'Slovenija', 'Česká republika', 'Cyprus', 'Türkiye',
       'Indonesia', 'Slovenská republika', '日本', 'Srbija', 'Chile',
       'Zimbabwe', 'Magyarország', 'دولة الكويت', 'Republic of Mozambique',
       'Eesti', 'Peru', '香港', 'Россия', 'Latvija', 'مصر', 'Bahamas',
       'Uganda', 'Sri Lanka', 'România', '新加坡', 'Republika ng Pilipinas',
       "Belarus'", 'دولة قطر

In [66]:
sum(tweetsDF.location.value_counts())

1673

### Topic Modeling with LDA

In [30]:
en.STOPWORDS.add('eu')
nlp = en.English()

In [31]:
cleanDF = pd.DataFrame(columns=['id', 'text'])

In [32]:
cleanDF['text'] = tweetsDF['text']
cleanDF.drop('id', axis=1)

,text
0,U.K.'s Cameron Appeals to the Undecided Ahead ...
1,"@PuppyMoof @YouTube Stay, PLEASE! I can get EU..."
2,EU referendum: The view from Colchester https:...
3,RT @BritainUnited1: WHY DOES SNP WANT TO BREAK...
4,U.K.'s Cameron Appeals to the Undecided Ahead ...
5,Now Streaming at https://t.co/V0SeDzX1VE\nleav...
6,U.K.'s Cameron Appeals to the Undecided Ahead ...
7,eu hzkzuxkxukzux https://t.co/kMCXKvywzU
8,RT @NatalieMcGarry: Last thought before bed. T...
9,Eu inteligente disse good good


In [33]:
def get_tokens(text):  
    """Return lemmatized tokens."""
    text = re.sub(r"http\S+", "", text) 
    text = re.sub("RT", "", text)
    text = re.sub("amp", "", text)
    letters_only = re.sub("[^a-zA-Z]", " ", text) 
    words = ' '.join(letters_only.lower().split())
    try:
        tokens = [token.lemma_ for token in nlp(words)] 
    except:
        tokens = [token.lemma_ for token in nlp(words.decode('utf8'))] 
    filtered = [t for t in tokens if t != '' and t != ' ' and t != '\n' and t != '\n\n']
    return ' '.join(filtered)

In [34]:
cleanDF['tokens'] = cleanDF['text'].apply(get_tokens)

In [35]:
def prep_pylda(docs, n_topics = 4):   
    vect = TfidfVectorizer(max_df = 0.5, max_features = 10000,
                                 min_df = 5, stop_words = STOPWORDS,
                                 use_idf = True, tokenizer = None, ngram_range=(1, 3))
    matrix = vect.fit_transform(docs)
    vocab = vect.get_feature_names()
    
    # fit transform lda
    lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                learning_method='online', learning_offset=50.,
                random_state=0, doc_topic_prior = .001)
    doc_topic_dists = lda.fit_transform(matrix)
    
    # prepare pyLDAvis stuff 
    get_normed = lambda data: pd.DataFrame(data).div(data.sum(axis = 1), axis = 0) 
    prepared = pyLDAvis.prepare(
            doc_lengths = docs.str.len(),
            vocab = vocab,
            term_frequency = np.asarray(matrix.sum(axis = 0)).ravel().tolist(),
            topic_term_dists = get_normed(lda.components_),  
            doc_topic_dists = get_normed(doc_topic_dists)) 
    
    return prepared

In [36]:
prepared = prep_pylda(cleanDF.tokens, n_topics = 4)

In [37]:
# display in notebook
pyLDAvis.display(prepared)

In [38]:
# export to html
pyLDAvis.save_html(prepared, 'european.html')

### Try Some Text Blobbing

In [40]:
def normalize_tweet(text):
    text = re.sub(r"http\S+", "", text) 
    text = re.sub("RT", "", text)
    text = re.sub("amp", "", text)
    letters_only = re.sub("[^a-zA-Z]", " ", text) 
    words = ' '.join(letters_only.lower().split())
    return words

In [54]:
cleanlist = []
bloblist = []
for tweet in cleanDF.text:
    cleantweet = normalize_tweet(tweet)
    cleanlist.append(cleantweet)
    blob = TextBlob(cleantweet)
    bloblist.append(blob)

In [55]:
sentlist = []
for blob in bloblist:
    bs = blob.sentiment
    sentlist.append(bs)

sentDF = pd.DataFrame()
sentDF['sentiment'] = sentlist

In [56]:
sentDF = sentDF['sentiment'].apply(pd.Series)

In [57]:
sentDF.rename(columns={0: 'polarity', 1: 'subjectivity'}, inplace=True)
print(sentDF.polarity.mean())
print(sentDF.subjectivity.mean())

0.06957474502008097
0.26492817879163655


In [ ]:
sentDF['tweet'] = cleanDF.text
sentDF.dropna(axis=0, inplace=True)
sentDF

### K-Means Clustering

In [42]:
my_words = set(['eu'])
my_stop_words = text.ENGLISH_STOP_WORDS.union(my_words)

In [51]:
true_k = 4
vectorizer = TfidfVectorizer(stop_words=set(my_stop_words))
X = vectorizer.fit_transform(cleanlist)
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=1000, n_init=1)
model.fit(X)
joblib.dump(model, 'doc_cluster2.pkl')

['doc_cluster2.pkl', 'doc_cluster2.pkl_01.npy', 'doc_cluster2.pkl_02.npy']

In [52]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])
    print()

Top terms per cluster:
Cluster 0:
 yemen
 saudi
 arms
 vote
 embargo
 arabia
 parliament
 war
 civilians
 airstrikes

Cluster 1:
 crisis
 migrant
 austria
 ambassador
 recalls
 greece
 amid
 divisions
 sharp
 bbcbreaking

Cluster 2:
 uk
 referendum
 brexit
 leave
 britain
 cameron
 deal
 european
 migration
 says

Cluster 3:
 stay
 leave
 wants
 vote
 want
 uk
 like
 britain
 johnson
 say



In [53]:
clusters = model.labels_.tolist()
frame = pd.DataFrame(clusters, columns=['clusters'])
frame['clusters'].value_counts()

2    145035
0     10277
1      4684
3      4410
Name: clusters, dtype: int64